<a href="https://colab.research.google.com/github/loriboi/nfo_tanker_scheduling_problem/blob/main/tanker_scheduling_problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install cplex
!pip install docplex

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 635.6/635.6 kB 9.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for docplex: filename=docplex-2.27.239-py3-none-any.whl size=674503 sha256=cb81335a970874b070cd973cb6a4f3927184f19a6611f4ac21413410296963a0
  Stored in directory: /root/.cache/pip/wheels/62/7c/db/cef9425e1cedbf45621545097eaaeed5efe07005bd8229dd74
Successfully built docplex


In [4]:
from docplex.mp.model import Model
#formulazioni di flusso con archi e con path

# Grafo delle spedizioni
#1p = 1', 1s=1''
grafo = {
    's':['1p','2p','3p','4p'],
    '1p':['1s'],
    '2p':['2s'],
    '3p':['3s'],
    '4p':['4s'],
    '2s':['t'],
    '4s':['t'],
    '3s':['2p','4p','t'],
    '1s':['4p','2p','t'],
}

# Definizione modello
def crea_modello(grafo):
    modello = Model('min_num_paths')

    # Variabili di decisione: indicano se un arco è selezionato
    percorsi = {(u, v): modello.binary_var(name=f'path_{u}_{v}') for u in grafo.keys() for v in grafo[u]}

    # Vincolo 1: almeno un percorso deve partire da 's'
    vincolo_s = modello.sum(percorsi['s', v] for v in grafo['s']) >= 1
    modello.add_constraint(vincolo_s, ctname='at_least_one_path_from_s')

    # Vincolo 2: ogni arco deve essere utilizzato (tranne quelli che partono da s e arrivano in t)
    for u in grafo.keys():
        for v in grafo[u]:
            if v!='t':
              vincolo = modello.sum(percorsi[u, v] for u in grafo.keys() if v in grafo[u]) == 1
              modello.add_constraint(vincolo, ctname=f'edge_usage_{u}_{v}')

    # Vincolo 3: solo un arco esce da ogni nodo diverso da 's'
    for nodo in grafo.keys():
        if nodo != 's':
            vincolo_uscita = modello.sum(percorsi[nodo, v] for v in grafo[nodo]) == 1
            modello.add_constraint(vincolo_uscita, ctname=f'only_one_edge_out_{nodo}')

    modello.minimize(modello.sum(percorsi['s', v] for v in grafo['s']))

    return modello, percorsi


# Risoluzione del modello
def risolvi_modello(grafo):
    modello, percorsi = crea_modello(grafo)
    modello.solve()
    return modello, percorsi

# Funzione per stampare i risultati
def stampa_risultati(modello, percorsi):
    print('path:', modello.objective_value)
    for u, v in percorsi:
        if percorsi[u, v].solution_value != 0:
              print(f'Arco {u} a {v} è selezionato')

def trova_percorsi(modello, percorsi, grafo):
    percorsi_tot = []

    for u, v in percorsi:
        if u == 's' and percorsi[u, v].solution_value == 1:
            percorso = [u]
            nodo_attuale = v
            while nodo_attuale != '':
                percorso.append(nodo_attuale)
                if nodo_attuale in grafo:
                    for next_node in grafo[nodo_attuale]:
                        if percorsi[nodo_attuale, next_node].solution_value == 1:
                            nodo_attuale = next_node
                            break
                else:
                    break
            percorsi_tot.append(percorso)

    return percorsi_tot


# Risoluzione del modello e stampa dei risultati
modello, percorsi = risolvi_modello(grafo)
stampa_risultati(modello, percorsi)

percorsi_trovati = trova_percorsi(modello, percorsi, grafo)
print("Percorsi trovati:", percorsi_trovati)

path: 2.0
Arco s a 1p è selezionato
Arco s a 3p è selezionato
Arco 1p a 1s è selezionato
Arco 2p a 2s è selezionato
Arco 3p a 3s è selezionato
Arco 4p a 4s è selezionato
Arco 2s a t è selezionato
Arco 4s a t è selezionato
Arco 3s a 4p è selezionato
Arco 1s a 2p è selezionato
Percorsi trovati: [['s', '1p', '1s', '2p', '2s', 't'], ['s', '3p', '3s', '4p', '4s', 't']]
